<a href="https://colab.research.google.com/github/mesogin/TIS2/blob/main/0302_HomeWork_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#XGBoost 모형


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 44.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB

In [ ]:
#수업시간에 다룬 와인 데이터를 대상으로 3가지 모형(1. 랜덤포레스트 분류기 모형 2. XGBoost 모형 3. LightGBM) 을 기반으로 gradio 앱으로 만들기
# 단, 각각의 모형을 drowdown 형태로 선택할 수 있게 UI 구성할 것.
# title, description, markdown 등 기타 엑세서리 요소 추가는 보넌스점수
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
xgb1 = XGBClassifier(tree_method='hist', random_state=42, max_depth=3)
scores1 = cross_validate(xgb1, train_input, train_target, return_train_score=True, n_jobs=-1)
xgb2 = XGBClassifier(tree_method='hist', random_state=42, max_depth=3)
scores2 = cross_validate(xgb2, train_input, train_target, return_train_score=True, n_jobs=-1)


print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [ ]:
xgb.fit(train_input, train_target)
xgb1.fit(train_input, train_target)
xgb2.fit(train_input, train_target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [ ]:
alcohol_min = wine['alcohol'].min()
alcohol_max = wine['alcohol'].max()
alcohol_mean = wine['alcohol'].mean()
sugar_min = wine['sugar'].min()
sugar_max = wine['sugar'].max()
sugar_mean = wine['sugar'].mean()
pH_min = wine['pH'].min()
pH_max = wine['pH'].max()
pH_mean = wine['pH'].mean()

In [ ]:
import gradio as gr
# 함수 정의
def to_wine_name(number):
  if number == 1:
    return "화이트 와인이네요! 생선요리와 함께 즐겨보세요!"
  else:
    return "레드와인입니다!🍷 고기요리와 잘 어울립니다!"

def square(model, alcohol, sugar, ph):
    input_arr = [[alcohol, sugar, ph]] # 2차원 배열로 전환
    if model == "deciontree_default_depth 모형":
        wine_label = xgb.predict(input_arr)[0]
        return to_wine_name(wine_label)
    elif model == "deciontree_max_depth=3":
        wine_label = xgb1.predict(input_arr)[0]
        return to_wine_name(wine_label)
    elif model == "deciontree_max_depth=4":
        wine_label = xgb1.predict(input_arr)[0]
        return to_wine_name(wine_label)
    else:
        return "도형을 선택하세요"

# 입력 인터페이스
inputs = [
    gr.inputs.Dropdown(["deciontree_default_depth 모형", "deciontree_max_depth=3", "deciontree_max_depth=4"], 
    label="어떤 모형을 선택하시겠어요?"),
    gr.inputs.Slider(minimum=alcohol_min, maximum=alcohol_max, default=alcohol_mean, label="알코올 도수"),
    gr.inputs.Slider(minimum=sugar_min, maximum=sugar_max, default=sugar_mean, label="당도"),
    gr.inputs.Slider(minimum=pH_min, maximum=pH_max, default=pH_mean, label="산도")
]

# 출력 인터페이스
output_label = gr.outputs.Label(label="와인은?...")

# 앱 타이틀 설정
title="🕵️‍♀️당신의 와인 종류를 알려드립니다.🍷"
description="와인의 종류를 알아보기 위한 그라디오 프로그램입니다.."

# 인터페이스 실행
dropdown_demo = gr.Interface(fn=square, inputs=inputs, outputs=output_label, title=title, description=description)

dropdown_demo.launch()

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:89: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(valu

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>